In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1264995/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [4]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
9,model_6_3_0,0.999963,0.992829,0.999951,0.999981,0.999974,8.279333,1599.550956,11.426776,12.197905,11.812341,0.018630,2.877383,0.999919,2.999879,21.772475,37.617861,"Hidden Size=[3], regularizer=0.2, learning_rat..."
14,model_6_2_21,0.999960,0.992861,0.999889,0.999256,0.999851,8.860174,1592.461855,21.166260,15.340273,18.253267,0.019482,2.976604,0.999913,3.103325,21.636867,37.482253,"Hidden Size=[3], regularizer=0.2, learning_rat..."
15,model_6_2_18,0.999960,0.992888,0.999893,0.999235,0.999853,8.899479,1586.422556,20.421393,15.756618,18.089005,0.019300,2.983199,0.999913,3.110200,21.628015,37.473400,"Hidden Size=[3], regularizer=0.2, learning_rat..."
16,model_6_2_22,0.999960,0.992862,0.999888,0.999248,0.999850,8.908232,1592.224497,21.403416,15.493342,18.448379,0.019467,2.984666,0.999913,3.111730,21.626048,37.471434,"Hidden Size=[3], regularizer=0.2, learning_rat..."
17,model_6_3_1,0.999960,0.993061,0.999971,0.999961,0.999965,8.921933,1547.817695,6.735024,25.057654,15.896339,0.018071,2.986960,0.999913,3.114122,21.622975,37.468361,"Hidden Size=[3], regularizer=0.2, learning_rat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,model_35_5_4,0.996463,0.991650,0.999836,0.999829,0.999832,788.884775,1862.499127,34.487458,39.587037,37.037247,0.056920,28.087093,1.001391,29.282818,156.658759,260.263205,"Hidden Size=[21], regularizer=0.05, learning_r..."
3179,model_35_5_3,0.996394,0.991627,0.999838,0.999834,0.999836,804.365530,1867.714629,33.914539,38.343416,36.128978,0.057556,28.361339,1.001419,29.568740,156.619892,260.224338,"Hidden Size=[21], regularizer=0.05, learning_r..."
3194,model_35_5_2,0.996323,0.991602,0.999840,0.999838,0.999839,820.273532,1873.275419,33.551334,37.534698,35.543016,0.058203,28.640418,1.001447,29.859700,156.580724,260.185169,"Hidden Size=[21], regularizer=0.05, learning_r..."
3208,model_35_5_1,0.996249,0.991575,0.999841,0.999839,0.999840,836.613343,1879.191236,33.410188,37.178112,35.294150,0.058854,28.924269,1.001476,30.155635,156.541276,260.145721,"Hidden Size=[21], regularizer=0.05, learning_r..."
